# Virtual ADC example
---


## Load the bitstream into the Programmable Logic

In [79]:
# Import the X-HEEP Python class
from pynq import x_heep as xh
# Load the X-HEEP bitstream
x_heep = xh()

## Generate a random array to use as input signal 

In [107]:
import numpy as np
# Set the seed for reproducibility (optional)
np.random.seed(43)

# Generate a random integer array
array_size = 32
rand_in = np.random.randint(0, 100, size=array_size, dtype=np.int32)

# Print the generated array
print("Generated Array:", rand_in)

# Save the array to a binary file
file_path = "/home/xilinx/x-heep-femu-sdk/sw/riscv/apps/virtual_adc_read/adc_in.bin"
data = rand_in.copy()
with open(file_path, 'wb') as file:
    data.byteswap(True).tofile(file)

Generated Array: [68 64 49 21 58 16 51 17 59 91 66 30 87 64 35 50 75 89 65 34 75 66 99  4
 36 76 88 47 32 74 60 73]
[68 64 49 21 58 16 51 17 59 91 66 30 87 64 35 50 75 89 65 34 75 66 99  4
 36 76 88 47 32 74 60 73]


## Compile the code and run the application
You can change the variable `SKIP_COMPILE` to 0 to avoid compiling each time.

In [111]:
from pynq import x_heep as xh
# This will automatically make accessible the changes made in the sdk
%reload_ext autoreload 

# Skip compiling if all you want to do is run the application
SKIP_COMPILE = 1

# Compile the application
if SKIP_COMPILE or x_heep.compile_app("virtual_adc_read"):
    adc = x_heep.init_adc_mem() # Init the ADC memory
    x_heep.reset_adc_mem(adc)   # Reset the ADC memory
    x_heep.write_adc_mem(adc)   # Write the ADC memory
    out, _ = x_heep.run_app()   # Run the application. Disregard the error output

✅ Return SUCCESS
 68, 64, 49, 21, 58, 16, 51, 17, 59, 91, 66, 30, 87, 64, 35, 50, 75, 89, 65, 34, 75, 66, 99, 04, 36, 76, 88, 47, 32, 74, 60, 73, 


### Check if the CPU's output is the same as the input

In [110]:
app_out = [int(x) for x in out.split(',') if x.strip()]
if app_out == list(rand_in): print("✅ Success")
else: print("❌ Failed")


[68, 64, 49, 21, 58, 16, 51, 17, 59, 91, 66, 30, 87, 64, 35, 50, 75, 89, 65, 34, 75, 66, 99, 4, 36, 76, 88, 47, 32, 74, 60, 73]
✅ Success
